In [ ]:
import os
import pandas as pd
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import harp
import cartopy.crs as ccrs
import time
from datetime import timedelta
import xarray
from tqdm import tqdm

home = '/Users/stijn/Documents/jaar 2/UC/urbancomputing_group2/'
TROPOMI = '/data/TROPOMI/'
# home = ''
# TROPOMI = 'data/TROPOMI/'

In [ ]:
#The call to regrid and merge the selected files for given area of interest.

coordinates = {'lon_min': 19.5, 'lon_max': 29.5, 'lat_min': 31.5, 'lat_max': 34.2}

operations = ";".join([
    
    "latitude>" + str(coordinates['lat_min']) + ";latitude<" + str(coordinates['lat_max']),
    "longitude>" + str(coordinates['lon_min']) + ";longitude<" + str(coordinates['lon_max']),
    "cloud_fraction < 0.5",
    #"surface_meridional_wind_velocity < 7",
    #"surface_zonal_wind_velocity < 7",
    "tropospheric_NO2_column_number_density_validity>50",
    "keep(latitude_bounds,longitude_bounds,datetime_start,datetime_length,tropospheric_NO2_column_number_density,tropospheric_NO2_column_number_density_uncertainty,surface_zonal_wind_velocity,surface_meridional_wind_velocity)",
    "derive(datetime_stop {time} [days since 2000-01-01])",
    "derive(datetime_start [days since 2000-01-01])",
    "exclude(datetime_length)",
    "bin_spatial(62,31.5,0.045,225,19.5,0.045)",# number of pixels in latitude, latitude min, latitude step, number of pixels in longitude, longitude min, longitude step
    "derive(tropospheric_NO2_column_number_density)",
    "derive(latitude {latitude})",
    "derive(longitude {longitude})",
    "derive(surface_zonal_wind_velocity)",
    "derive(surface_meridional_wind_velocity)",
])

reduce_operations = "squash(time, (latitude, longitude, latitude_bounds, longitude_bounds));bin()" # squash time dimension 

In [ ]:
# for low let's regrid them one by one?

NO2_fp = home+TROPOMI
NO2_fns = sorted(glob(NO2_fp+'*/*.nc'))


if not os.path.exists(home+'/data/regridded_tropomi'): # create folder for regridded data
    os.makedirs(home+'/data/regridded_tropomi')

for filename in tqdm(NO2_fns):
    try:
        regridded = harp.import_product(filename, operations, reduce_operations=reduce_operations)
    except harp.NoDataError:
        print('No valid data for this file (in the given longitude, latitude range). Moving on to next file.')
        continue
    try:
        # harp.export_product(regridded, home+f'/data/regridded_tropomi/{regridded.source_product[:-3]}_regrid.nc', file_format='netcdf')
        harp.export_product(regridded, home+f'data/regridded_tropomi/{regridded.source_product[:-3]}_regrid.nc', file_format='netcdf')
    except harp.CLibraryError:
        print('File already exists and is in memory so cannot be rewritten. Moving on to next file.')
        pass

    # Plotting the regridded data
    # xds = xarray.open_dataset(home+f'/data/regridded_tropomi/{regridded.source_product[:-3]}_regrid.nc', engine='netcdf4')
    xds = xarray.open_dataset(home+f'data/regridded_tropomi/{regridded.source_product[:-3]}_regrid.nc', engine='netcdf4')
'''    fig,ax = plt.subplots(1,1,figsize=(10, 5))
    xds.tropospheric_NO2_column_number_density.plot(cmap='viridis', vmin=0,ax=ax)
    time = str(xds.datetime_start.values[0])[:19]
    ax.set_title('Tropospheric NO2 on ' + time,fontsize=20)
    plt.tight_layout()
    plt.savefig(f'figures/regrids/{time}.jpg')
    plt.close()
    xds.close()'''
